In [1]:
import sys
import spacy
import re
import pickle
import numpy as np
import pandas as pd
import scipy as sp
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from spacy.lang.en.stop_words import STOP_WORDS 
from collections import Counter
from plotnine import *
from pandas.tseries.offsets import MonthBegin
from yellowbrick.features import Rank2D
import feather
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTEENN
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from yellowbrick.cluster import SilhouetteVisualizer
from yellowbrick.cluster import KElbowVisualizer
import pickle
from yellowbrick.text import FreqDistVisualizer
from yellowbrick.text import TSNEVisualizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from yellowbrick.model_selection import LearningCurve
from yellowbrick.classifier import ConfusionMatrix, ROCAUC
from sklearn.preprocessing import StandardScaler
from yellowbrick.model_selection import ValidationCurve
from sklearn.metrics import classification_report

In [2]:
class EstimatorSelectionHelper:
    
    def __init__(self, models, params, params2, proportion_iterations):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError("Some estimators are missing parameters: %s" % missing_params)
        self.models = models
        self.params = params
        self.params2 = params2
        self.keys = models.keys()
        self.grid_searches = {}
        self.proportion_iterations = proportion_iterations
    
    def fit(self, X, y, cv=3, n_jobs=-1, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print("Running GridSearchCV for {}.".format(key))
            model = self.models[key]
            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs, 
                              verbose=verbose, scoring=scoring, refit=refit,
                              return_train_score=True)
            gs.fit(X,y)
            self.grid_searches[key] = gs
            
            
    
    def randomized_fit(self, X, y, cv=3, n_jobs=-1, verbose=1, scoring='f1'):
 
        params = self.params
        param_labels = []
        df = list(pd.DataFrame(self.params2).prod())
        
        for key in self.keys:
            param_labels.append(key)
        
        zipped = dict(zip(param_labels, df))
        
        for key in self.keys:
            print(key)
            if str(key) == 'RandomForestClassifier':
                n_iter = int(round(zipped[key]*(self.proportion_iterations**4.5)))
                print("Running RandomizedSearchCV for {} with {} iterations".format(key, n_iter))
            else:
                n_iter = int(round(zipped[key]*self.proportion_iterations)) 
                print("Running RandomizedSearchCV for {} with {} iterations".format(key, n_iter))
            model=self.models[key]
            params = self.params[key]
            rs = RandomizedSearchCV(model, params, cv=cv, n_jobs=n_jobs, verbose=verbose, scoring=scoring, return_train_score=True, n_iter=n_iter)
            rs.fit(X,y)
            self.grid_searches[key] = rs
       
    
    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                 'estimator': key,
                 'min_score': min(scores),
                 'max_score': max(scores),
                 'mean_score': np.mean(scores),
                 'std_score': np.std(scores),
            }
            return pd.Series({**params,**d})
                      
        rows = []
        for k in self.grid_searches:
            print(k)
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]        
                scores.append(r.reshape(len(params),1))

            all_scores = np.hstack(scores)
            for p, s in zip(params,all_scores):
                rows.append((row(k, s, p)))
        
        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)
        
        columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]
        
        return df[columns]


In [3]:
# Read in data
pitchfork_data = feather.read_dataframe('/home/michelle/Documents/data projects/pitchfork_data_analysis/data/processed/pitchfork_tfidf_aug30.feather')
#pitchfork_data = pd.read_csv('/home/michelle/Documents/data projects/pitchfork_data_analysis/data/processed/pitchfork_tfidf_aug30.csv')

In [4]:
pitchfork_data['category'] = pitchfork_data['category'].fillna(0)
pitchfork_data.head()

,publication_date,author,artist_name,album_x,label,score,review,like,song,album_1,...,shudder,sear,poke,hawk,dfa,torn,gibbard,juxtapos,album_y,category
0,2018-08-28 05:00:00,Evan Rytlewski,Devon Welsh,Dream Songs,You Are Accepted,6.9,For a songwriter who’s always been fixated on ...,0.131949,0.107358,0.021111,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0
1,2018-08-28 05:00:00,Philip Sherburne,Bamba Pana,Poaa,Nyege Nyege Tapes,7.2,"At first glance, Kampala, Uganda’s Nyege Nyege...",0.088597,0.028834,0.014175,...,0.065385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0
2,2018-08-28 05:00:00,Dean Van Nguyen,Nef the Pharaoh,The Big Chang Theory,KILFMB,7.3,Nef the Pharaoh takes the weight of his hometo...,0.080508,0.018341,0.036066,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0
3,2018-08-28 05:00:00,Stuart Berman,Matthew “Doc” Dunn,Lightbourn,Cosmic Range,8.0,"For over a decade now, Matthew “Doc” Dunn has ...",0.116932,0.016649,0.049109,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0
4,2018-08-27 05:00:00,Ian Cohen,Interpol,Marauder,Matador,6.1,More than any of their frenemies in the 2000s ...,0.046965,0.010699,0.052599,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Antics,1.0


In [5]:
# Get x-y values
X=pitchfork_data.iloc[:,7:-2]
y = pitchfork_data.category.values

In [6]:
del pitchfork_data

In [ ]:

scaler = StandardScaler()
X  = scaler.fit_transform(X,y)

In [7]:
# Test-Train Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=np.random.randint(1,1000))

In [8]:
# Oversample because class imbalance and also dimensionality reasons.
X_res, y_res=SMOTE().fit_sample(X_train,y_train)

In [9]:
models1 = {
    #'MultinomialNB': MultinomialNB(),
    'RandomForestClassifier': RandomForestClassifier(),
    #'KNeighborsClassifier': KNeighborsClassifier()
}

params1 = {
    #'MultinomialNB':{'alpha': np.linspace(0.5,20,50),
      #              'fit_prior': [True],
      #              'class_prior': [None]},
    'RandomForestClassifier': {'n_estimators':np.arange(1,650,40),
             'max_depth': np.arange(2,31,2),
             'min_samples_leaf': np.arange(2,31,1),
             'min_samples_split': np.arange(2,31,1)},
    #'KNeighborsClassifier': {'n_neighbors': np.arange(2,10,1)}
}

params2 = {
    #'MultinomialNB':{'alpha': len(np.linspace(0.5,20,50)),
      #              'fit_prior': len([True]),
      #              'class_prior': len([None])},
    'RandomForestClassifier': {'n_estimators':len(np.arange(1,650,40)),
             'max_depth': len(np.arange(2,31,2)),
             'min_samples_leaf': len(np.arange(2,31,1)),
             'min_samples_split': len(np.arange(2,31,1))},
    #'KNeighborsClassifier': {'n_neighbors': len(np.arange(2,16,1))}
}



In [13]:
X

,like,song,album_1,band,sound,music,record,track,guitar,rock,...,mp3,shortcom,shudder,sear,poke,hawk,dfa,torn,gibbard,juxtapos
0,0.131949,0.107358,0.021111,0.055243,0.042117,0.112961,0.068587,0.023139,0.028090,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.0,0.000000
1,0.088597,0.028834,0.014175,0.000000,0.197955,0.075847,0.015351,0.031073,0.000000,0.000000,...,0.000000,0.0,0.065385,0.0,0.000000,0.0,0.0,0.00000,0.0,0.000000
2,0.080508,0.018341,0.036066,0.000000,0.017988,0.057895,0.078116,0.039530,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.0,0.000000
3,0.116932,0.016649,0.049109,0.042836,0.000000,0.017518,0.017728,0.017942,0.043563,0.023551,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.0,0.000000
4,0.046965,0.010699,0.052599,0.055055,0.031481,0.000000,0.022785,0.000000,0.000000,0.060538,...,0.047998,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.0,0.000000
5,0.093361,0.017724,0.017427,0.000000,0.104300,0.093247,0.000000,0.019100,0.069563,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.0,0.000000
6,0.161456,0.000000,0.016438,0.043016,0.032795,0.087960,0.017802,0.018017,0.065619,0.047300,...,0.000000,0.0,0.000000,0.0,0.074428,0.0,0.0,0.00000,0.0,0.000000
7,0.103580,0.026219,0.012889,0.000000,0.064287,0.055175,0.013959,0.098892,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.0,0.000000
8,0.103580,0.026219,0.012889,0.000000,0.064287,0.055175,0.013959,0.098892,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.0,0.000000
9,0.103580,0.026219,0.012889,0.000000,0.064287,0.055175,0.013959,0.098892,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.0,0.000000


In [ ]:
models1 = {
    'RandomForestClassifier': RandomForestClassifier(),
    'MultinomialNB': MultinomialNB(),
    'LogisticRegression': linear_model.LogisticRegression(),
    'SVC': SVC(),
    'KNeighborsClassifier': KNeighborsClassifier()
}

params1 = {
    'RandomForestClassifier': {'n_estimators':np.arange(1,650,50),
             'max_depth': np.arange(2,31,2),
             'min_samples_leaf': np.arange(2,31,1),
             'min_samples_split': np.arange(2,31,1)},
    'MultinomialNB':{'alpha': np.linspace(0.5,1,10),
                    'fit_prior': [True],
                    'class_prior': [None]},
    'LogisticRegression':{'C': np.logspace(0, 4, 15),
                         'penalty':['l1','l2']},
    'SVC': {'C': np.linspace(0.5,1,10),
           'kernel': ['linear', 'poly', 'rbf']},
    'KNeighborsClassifier': {'n_neighbors': np.arange(2,10,1)}
}

params2 = {
    'RandomForestClassifier': {'n_estimators':len(np.arange(1,650,50)),
             'max_depth': len(np.arange(2,31,2)),
             'min_samples_leaf': len(np.arange(2,31,1)),
             'min_samples_split': len(np.arange(2,31,1))},
    
    'MultinomialNB':{'alpha': len(np.linspace(0.5,1,10)),
                    'fit_prior': len([True]),
                    'class_prior': len([None])},
    
    'LogisticRegression':{'C': len(np.logspace(0, 4, 15)),
                         'penalty':len(['l1','l2'])},
    
             
    'SVC': {'C': len(np.linspace(0.5,1,10)),
           'kernel': len(['linear', 'poly', 'rbf'])},
    
    'KNeighborsClassifier': {'n_neighbors': len(np.arange(2,16,1))}
}



In [11]:
helper1 = EstimatorSelectionHelper(models1, params1, params2, proportion_iterations=.15)
helper1.randomized_fit(X_res, y_res, scoring='f1', n_jobs=-1, cv=5)
#helper1.randomized_fit(X_train, y_train, scoring='f1', n_jobs=-1, cv=5)

RandomForestClassifier
Running RandomizedSearchCV for RandomForestClassifier with 42 iterations
Fitting 5 folds for each of 42 candidates, totalling 210 fits


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/home/michelle/anaconda3/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/home/michelle/anaconda3/li

In [ ]:
helper1.score_summary(sort_by='mean_score').to_csv('model_scores_aug30.csv', index=False)

In [ ]:
helper1.score_summary(sort_by='mean_score')

In [ ]:
clf = RandomForestClassifier(n_estimators=551,min_impurity_split=27, min_samples_leaf=18, max_depth=20)
clf = clf.fit(X_train,y_train)
pred = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, pred))

### Diagnostics

In [ ]:
fig = plt.figure(figsize=(8,8), dpi=250)
cv = StratifiedKFold(12)
sizes = np.linspace(0.3, 1.0, 10)

viz = LearningCurve(
    RandomForestClassifier(n_estimators=81,min_samples_split=2, min_samples_leaf=5, max_depth=30), cv=cv, train_sizes=sizes,
    scoring='accuracy', n_jobs=4
)

# Fit and poof the visualizer
viz.fit(X, y)
viz.poof()

In [ ]:
fig = plt.figure(figsize=(8,8), dpi=250)
visualizer = ROCAUC(RandomForestClassifier(n_estimators=81,min_samples_split=2, min_samples_leaf=5, max_depth=30))

visualizer.fit(X_train, y_train)  # Fit the training data to the visualizer
visualizer.score(X_test, y_test)  # Evaluate the model on the test data
g = visualizer.poof()   

In [ ]:
# The ConfusionMatrix visualizer taxes a model
fig = plt.figure(figsize=(8,8), dpi=250)
cm = ConfusionMatrix(RandomForestClassifier(n_estimators=81,min_samples_split=2, min_samples_leaf=5, max_depth=30), percent=True)

# Fit fits the passed model. This is unnecessary if you pass the visualizer a pre-fitted model
cm.fit(X_train, y_train)

# To create the ConfusionMatrix, we need some test data. Score runs predict() on the data
# and then creates the confusion_matrix from scikit-learn.
cm.score(X_test, y_test)

# How did we do?
cm.poof()

### Text Viz

In [ ]:
corpus = pickle.load( open( "/home/michelle/Documents/data projects/pitchfork_data_analysis/data/processed/corpus.p", "rb" ) )

In [ ]:
fig = plt.figure(figsize=(8,25), dpi=250)

vectorizer = CountVectorizer()
docs       = vectorizer.fit_transform(corpus)
features   = vectorizer.get_feature_names()

visualizer = FreqDistVisualizer(features=features, n=100)
visualizer.fit(docs)
visualizer.poof()

### Clustering (Kmeans and TNSE)

In [ ]:
fig = plt.figure(figsize=(8,8), dpi=250)
visualizer = KElbowVisualizer(KMeans(), k=(2,10), metric='distortion')
visualizer.fit(X_train) # Fit the training data to the visualizer
visualizer.poof() # Draw/show/poof the data

In [ ]:
# Instantiate the clustering model and visualizer
fig = plt.figure(figsize=(15,15), dpi=275)
model = KMeans(6)
visualizer = SilhouetteVisualizer(model)

visualizer.fit(X_train) # Fit the training data to the visualizer
visualizer.poof() # Draw/show/poof the data

In [ ]:
fig = plt.figure(figsize=(16,16), dpi=275)
tfidf  = TfidfVectorizer()
docs   = tfidf.fit_transform(corpus)
tsne = TSNEVisualizer(labels=["documents"], decompose='svd')
tsne.fit(docs)
tsne.poof()